In [1]:
#|default_exp app

In [2]:
#|export
from fastai.text.all import *
import gradio as gr

In [9]:
#|export
import pathlib
plt = platform.system()
if plt == 'Linux': pathlib.WindowsPath = pathlib.PosixPath
if plt == 'Windows': pathlib.PosixPath = pathlib.WindowsPath

In [4]:
path = Path()

In [5]:
path.ls(file_exts='.pkl')

(#3) [Path('WriterHP_model_10ep.pkl'),Path('WriterStKng_model_10ep.pkl'),Path('WriterW_model_10ep.pkl')]

In [6]:
class SpacyTokenizerBig():
# Измененный класс токенайзера для поддержки словаря большого размера
    def __init__(self, lang='en', special_toks=None, buf_sz=5000):
        import spacy
        from spacy.symbols import ORTH
        self.special_toks = ifnone(special_toks, defaults.text_spec_tok)
        nlp = spacy.blank(lang)
        nlp.max_length = 10**10
        for w in self.special_toks: nlp.tokenizer.add_special_case(w, [{ORTH: w}])
        self.pipe,self.buf_sz = nlp.pipe,buf_sz

    def __call__(self, items):
        return (L(doc).attrgot('text') for doc in self.pipe(map(str,items), batch_size=self.buf_sz))

In [10]:
#|export
learnHP = load_learner('WriterHP_model_10ep.pkl')
learnStK = load_learner('WriterStKng_model_10ep.pkl')
learnW = load_learner('WriterW_model_10ep.pkl')

In [13]:
#|export
def write_text(text, author, words_cnt):
    words_cnt=int(words_cnt)
    if author == 'Harry Potter Style':
        preds = [learnHP.predict(text, words_cnt, temperature=1)]
    elif author == 'Stephen King Style':
        preds = [learnStK.predict(text, words_cnt, temperature=1)]
    elif author == 'Harry Potter Style + Stephen King Style':
        preds = [learnW.predict(text, words_cnt, temperature=1)]
    new_text = "\n".join(preds)
    return new_text

In [14]:
#|export
input_text = gr.components.Textbox(value='It was a bright day.', label='Input text')
label = gr.components.Textbox()
wordcount_num = gr.components.Number(value=40, label='New words count')
author_radio = gr.components.Radio(choices=["Harry Potter Style",
                                                                "Stephen King Style",
                                                                "Harry Potter Style + Stephen King Style"],
                                  value="Harry Potter Style",
                                  label='Author')

intface = gr.Interface(fn=write_text, inputs=[input_text, author_radio, wordcount_num], outputs=label)
intface.launch(inline=False)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Экспорт файла HotdogOrNot_APP.ipynb для загрузки на Hugging Face
from nbdev.export import nb_export
nb_export('HotdogOrNot_APP.ipynb','./')